In [4]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import numpy as np


In [5]:
device_u = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device_T = torch.device('cuda:1' if torch.cuda.device_count() > 1 else device_u)

print(device_u)
print(device_T)

cuda:0
cuda:1


In [6]:
class FCN(nn.Module):
    def __init__(self, in_dim, out_dim, hidden_layers=4, neurons=64):
        super(FCN, self).__init__()
        layers = [nn.Linear(in_dim, neurons), nn.Tanh()]
        for _ in range(hidden_layers - 1):
            layers += [nn.Linear(neurons, neurons), nn.Tanh()]
        layers.append(nn.Linear(neurons, out_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

net_u = FCN(3, 1).to(device_u)  # For u(x, y, t)
net_T = FCN(3, 1).to(device_T)  # For T(x, y, t)

In [7]:
def generate_collocation_points(N):
    x = np.random.rand(N, 1)
    y = np.random.rand(N, 1)
    t = np.random.rand(N, 1)
    X = np.hstack((x, y, t))
    return torch.tensor(X, dtype=torch.float32, requires_grad=True)

N_f = 10000
X_f = generate_collocation_points(N_f)
X_f_u = X_f.clone().detach().to(device_u).requires_grad_(True)
X_f_T = X_f.clone().detach().to(device_T).requires_grad_(True)

In [8]:
def compute_pde_u(X):
    u = net_u(X)
    grads = autograd.grad(u, X, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_t = grads[:, 2:3]
    u_x = grads[:, 0:1]
    u_xx = autograd.grad(u_x, X, grad_outputs=torch.ones_like(u_x), create_graph=True)[0][:, 0:1]
    
    nu = 0.01
    res_u = u_t + u * u_x - nu * u_xx
    return res_u

def compute_pde_T(X):
    X_u = X.to(device_u).detach().clone().requires_grad_(True)
    with torch.no_grad():
        u = net_u(X_u).to(device_T)
    
    T = net_T(X)
    grads = autograd.grad(T, X, grad_outputs=torch.ones_like(T), create_graph=True)[0]
    T_t = grads[:, 2:3]
    T_xx = autograd.grad(grads[:, 0:1], X, grad_outputs=torch.ones_like(grads[:, 0:1]), create_graph=True)[0][:, 0:1]
    T_yy = autograd.grad(grads[:, 1:2], X, grad_outputs=torch.ones_like(grads[:, 1:2]), create_graph=True)[0][:, 1:2]

    alpha = 0.01
    gamma = 1.0
    Q = gamma * u ** 2
    res_T = T_t - alpha * (T_xx + T_yy) - Q
    return res_T

In [9]:
def loss_function():
    res_u = compute_pde_u(X_f_u)
    res_T = compute_pde_T(X_f_T)
    loss_u = torch.mean(res_u ** 2)
    loss_T = torch.mean(res_T ** 2)
    return loss_u, loss_T

In [10]:
optimizer_u = torch.optim.Adam(net_u.parameters(), lr=1e-3)
optimizer_T = torch.optim.Adam(net_T.parameters(), lr=1e-3)

In [11]:
for epoch in range(5000):
    optimizer_u.zero_grad()
    optimizer_T.zero_grad()

    loss_u, loss_T = loss_function()
    total_loss = loss_u + loss_T.to(device_u)

    loss_u.backward(retain_graph=True)
    loss_T.backward()

    optimizer_u.step()
    optimizer_T.step()

    if epoch % 500 == 0:
        print(f"Epoch {epoch}: Loss_u = {loss_u.item():.4e}, Loss_T = {loss_T.item():.4e}, Total = {total_loss.item():.4e}")

/usr/lib/python3/dist-packages/torch/autograd/graph.py:823: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ./aten/src/ATen/cuda/CublasHandlePool.cpp:180.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 0: Loss_u = 4.1478e-04, Loss_T = 4.9559e-05, Total = 4.6434e-04
Epoch 500: Loss_u = 3.9894e-08, Loss_T = 8.1209e-09, Total = 4.8015e-08
Epoch 1000: Loss_u = 1.5027e-08, Loss_T = 3.7634e-09, Total = 1.8791e-08
Epoch 1500: Loss_u = 7.2254e-09, Loss_T = 2.1708e-09, Total = 9.3962e-09
Epoch 2000: Loss_u = 4.2561e-09, Loss_T = 1.2998e-09, Total = 5.5559e-09
Epoch 2500: Loss_u = 2.6126e-09, Loss_T = 8.4979e-10, Total = 3.4624e-09
Epoch 3000: Loss_u = 3.2983e-09, Loss_T = 5.9160e-10, Total = 3.8899e-09
Epoch 3500: Loss_u = 1.5568e-09, Loss_T = 4.5268e-10, Total = 2.0095e-09
Epoch 4000: Loss_u = 1.6891e-09, Loss_T = 3.6278e-10, Total = 2.0519e-09
Epoch 4500: Loss_u = 7.7713e-10, Loss_T = 4.2438e-10, Total = 1.2015e-09
